In [1]:
from transformers import PreTrainedTokenizerFast, GPT2LMHeadModel, AutoTokenizer, AutoModelForCausalLM
import torch

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [3]:
model = AutoModelForCausalLM.from_pretrained("output/merge_lora").to(device)
tokenizer = AutoTokenizer.from_pretrained("output/merge_lora")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
model = GPT2LMHeadModel.from_pretrained("gunwoo723/kogpt-trinity-poem-generator").to(device)
tokenizer = PreTrainedTokenizerFast.from_pretrained("gunwoo723/kogpt-trinity-poem-generator",
  bos_token='<s>', eos_token='</s>', unk_token='<unk>',
  pad_token='<pad>', mask_token='<mask>')

In [ ]:
tokenizer.add_special_tokens({'additional_special_tokens': ["<yun>"]})
model.resize_token_embeddings(len(tokenizer))

In [15]:
sentence = '[BOS]멈춰버린 시계 속 마지막으로 남은 바램\n'
inputs = tokenizer(sentence, add_special_tokens=True, return_tensors='pt').to(device)

with torch.no_grad():
    output = model.generate(
        input_ids=inputs["input_ids"],
        temperature=1.0, # 생성 다양성 조절
        min_length=64, # 생성되는 문장의 최소 길이
        max_length=128, # 생성되는 문장의 최대 길이
        repetition_penalty=1.2, # 반복을 줄이는 패널티
        do_sample=True, # 샘플링 기반 생성 활성화
        early_stopping=True, # EOS token을 만나면 조기 종료
        eos_token_id=tokenizer.eos_token_id,
        pad_token_id=tokenizer.pad_token_id,
        bos_token_id=tokenizer.bos_token_id,
        attention_mask=inputs["attention_mask"]
    )

generated_sequence = output[0].tolist()
text = tokenizer.decode(generated_sequence, skip_special_tokens=False)

In [16]:
text = text.replace("[YUN]", "\n").replace("[BOS]", "").replace("[EOS]", "")

In [17]:
print(text)

멈춰버린 시계 속 마지막으로 남은 바램
'세상의 빛'이 되다.
시간을 지배하는 자. 절대권력자가 되었던 전설적 CEO, 이 사람! 지금도 당신에게 있어 '1년'이란 의미의 숫자는 어떤 것일까.. 아마도 한 해의 시작에 앞서 혹은 새해의 각오를 다짐하고 결의할 때 말이다. 1이란 글자가 가진 특별한 느낌과 상징성이 우리들의 마음속에서 일고 있음은 부정할 수 없으리라 생각한다. 하지만 그 1 자체의 절대성과 유일성으로 인해 인간 삶속에서 '시간' 개념의
